In [97]:
import pandas as pd
import unicodedata
import requests
from datetime import datetime as dt

from bs4 import BeautifulSoup as soup
from helpers import regression as rh
from helpers.file import load_from_file
from services import MySQLService

In [98]:
season = 2024
target_stats = ["pts", "reb", "ast"]

mysql = MySQLService()

In [99]:
models_info = dict()

for target_stat in target_stats:
    model_info = load_from_file("../dist/betting", target_stat)

    print("\nModel name:", model_info["name"])
    print("Model features:", model_info["features"])
    print("Model stats_columns:", model_info["stats_columns"])
    print("Model confidence margin:", model_info["confidence_margin"])
    print("Model standard deviation:", model_info["std"])

    models_info[target_stat] = model_info


Model name: GradientBoosting
Model features: ['pts_avg', 'opponent_avg_conceded_pts', 'days_since_last_game', 'fg_pct_avg', 'opponent_avg_conceded_fg_pct', 'fg3_pct_avg', 'opponent_avg_conceded_fg3_pct']
Model stats_columns: ['pts', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']
Model confidence margin: 4.749815922952678
Model standard deviation: 6.147324213165769

Model name: GradientBoosting
Model features: ['reb_avg', 'opponent_avg_conceded_reb', 'days_since_last_game', 'oreb_avg', 'opponent_avg_conceded_oreb', 'dreb_avg', 'opponent_avg_conceded_dreb']
Model stats_columns: ['pts', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']
Model confidence margin: 1.9577611111568531
Model standard deviation: 2.5517405214834064

Model name: GradientBoosting
Model features: ['ast_avg', 'opponent_avg_conceded_ast', 'days_since_las

In [100]:
def remove_accents(input_str):
    return ''.join(
        c for c in unicodedata.normalize('NFD', input_str)
        if unicodedata.category(c) != 'Mn'
    )

season_games_plyrs = mysql.execute_query(f"""
    SELECT g.id as game_id, g.date, g.season, g.is_playoff, g.winner, g.home_id, p.name as player_name, g.away_id, pg.team_id, pg.player_id, pg.minutes, pg.pts, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.tov, pg.pf, pg.plus_minus
    FROM player_games AS pg
        LEFT JOIN games as g on pg.game_id = g.id 
        LEFT JOIN players as p on pg.player_id = p.id
    WHERE g.season = 2024
    ORDER BY g.date ASC""")

teams_df = mysql.get_data("teams")

order_by_clause = "date ASC"
games_df = mysql.get_data("games", order_by_clause=order_by_clause)

stats_columns = ['pts','fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta',
    'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']

rh.build_avg_columns(season_games_plyrs, stats_columns)

rh.build_days_since_last_game_col(season_games_plyrs)

season_games_plyrs = rh.build_opponent_conceded_columns(season_games_plyrs, games_df, stats_columns)

season_games_plyrs["player_name"] = season_games_plyrs["player_name"].str.replace(" Jr.", "")
season_games_plyrs['player_name'] = season_games_plyrs['player_name'].apply(remove_accents)

display(season_games_plyrs)

c:\Users\caiog\Projects\Personal\NBA-Predictor\src\services\mysql.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resp = pd.read_sql_query(query, db)
100%|██████████| 18/18 [00:03<00:00,  5.97it/s]


,game_id,date,season,is_playoff,winner,home_id,player_name,away_id,team_id_x,player_id,...,opponent_avg_conceded_fta,opponent_avg_conceded_ft_pct,opponent_avg_conceded_oreb,opponent_avg_conceded_dreb,opponent_avg_conceded_reb,opponent_avg_conceded_ast,opponent_avg_conceded_stl,opponent_avg_conceded_blk,opponent_avg_conceded_tov,opponent_avg_conceded_pf
0,22400061,2024-10-22,2024,0,H,1610612738,Tyler Kolek,1610612752,1610612752,1642278,...,16.2,0.7482,9.6,31.9,41.5,22.6,5.6,2.4,13.0,18.9
1,22400062,2024-10-22,2024,0,H,1610612747,Bronny James,1610612750,1610612747,1642355,...,19.7,0.8202,9.0,31.3,40.3,22.7,5.9,5.6,12.1,19.5
2,22400061,2024-10-22,2024,0,H,1610612738,Pacome Dadiet,1610612752,1610612752,1642359,...,16.2,0.7482,9.6,31.9,41.5,22.6,5.6,2.4,13.0,18.9
3,22400061,2024-10-22,2024,0,H,1610612738,Miles McBride,1610612752,1610612752,1630540,...,16.2,0.7482,9.6,31.9,41.5,22.6,5.6,2.4,13.0,18.9
4,22400062,2024-10-22,2024,0,H,1610612747,Austin Reaves,1610612750,1610612747,1630559,...,19.7,0.8202,9.0,31.3,40.3,22.7,5.9,5.6,12.1,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8744,22400374,2024-12-20,2024,0,H,1610612739,Craig Porter,1610612749,1610612739,1641854,...,23.9,0.8197,10.9,32.0,42.9,24.6,8.5,4.5,12.4,20.8
8745,22400374,2024-12-20,2024,0,H,1610612739,Liam Robbins,1610612749,1610612749,1641857,...,22.3,0.8262,11.0,35.9,46.9,25.1,7.9,5.3,16.3,21.0
8746,22400373,2024-12-20,2024,0,H,1610612755,Tidjane Salaun,1610612766,1610612766,1642275,...,20.4,0.8153,9.9,31.3,41.2,26.2,8.2,5.9,17.3,20.1
8747,22400374,2024-12-20,2024,0,H,1610612739,Jaylon Tyson,1610612749,1610612739,1642281,...,23.9,0.8197,10.9,32.0,42.9,24.6,8.5,4.5,12.4,20.8


In [101]:
season_games_plyrs.columns

Index(['game_id', 'date', 'season', 'is_playoff', 'winner', 'home_id',
       'player_name', 'away_id', 'team_id_x', 'player_id', 'minutes', 'pts',
       'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta',
       'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'plus_minus', 'pts_avg', 'fgm_avg', 'fga_avg', 'fg_pct_avg', 'fg3m_avg',
       'fg3a_avg', 'fg3_pct_avg', 'ftm_avg', 'fta_avg', 'ft_pct_avg',
       'oreb_avg', 'dreb_avg', 'reb_avg', 'ast_avg', 'stl_avg', 'blk_avg',
       'tov_avg', 'pf_avg', 'days_since_last_game', 'opponent_team_id',
       'team_id_y', 'opponent_avg_conceded_pts', 'opponent_avg_conceded_fgm',
       'opponent_avg_conceded_fga', 'opponent_avg_conceded_fg_pct',
       'opponent_avg_conceded_fg3m', 'opponent_avg_conceded_fg3a',
       'opponent_avg_conceded_fg3_pct', 'opponent_avg_conceded_ftm',
       'opponent_avg_conceded_fta', 'opponent_avg_conceded_ft_pct',
       'opponent_avg_conceded_oreb', 'opponent_avg_conceded

In [104]:
def get_games_info():
    url = "https://www.rotowire.com/basketball/nba-lineups.php"
    result = requests.get(url)
    doc = soup(result.text, "html.parser")
    games = doc.find_all("div", {"class": "lineup__box"})
    
    for g in games:
        try:
            game_dict = {
                'season': season,
                'date': dt.now()
            }
            
            teams = [x.text for x in g.find_all("div", {"class": "lineup__abbr"})]
            if len(teams) == 0:
                continue

            game_dict["away_abbv"] = teams[0]
            game_dict["home_abbv"] = teams[1]

            home_team = teams_df.loc[teams_df['abbreviation'] == game_dict["home_abbv"]].iloc[0]
            away_team = teams_df.loc[teams_df['abbreviation'] == game_dict["away_abbv"]].iloc[0]
            
            game_dict["away_id"] = away_team["id"]
            game_dict["home_id"] = home_team["id"]

            away_lineup = g.find("ul", {"class": "lineup__list is-visit"})
            home_lineup = g.find("ul", {"class": "lineup__list is-home"})

            away_lineup = [x.text.split('\n')[-2] for x in away_lineup.find_all("li", {"class": "lineup__player"})][:5]
            home_lineup = [x.text.split('\n')[-2] for x in home_lineup.find_all("li", {"class": "lineup__player"})][:5]

            home_players_df = rh.build_next_games_df(home_lineup, home_team, away_team, season_games_plyrs, stats_columns, games_df)
            away_players_df = rh.build_next_games_df(away_lineup, away_team, home_team, season_games_plyrs, stats_columns, games_df)

            merged_df = pd.concat([home_players_df, away_players_df], ignore_index=True)

            target_stat_columns = []

            for target_stat in models_info:
                model_info = models_info[target_stat]
                merged_df[f'pred_{target_stat}'] = model_info["model"].predict(merged_df[model_info["features"]])

                merged_df[f'lower_bound_{target_stat}'] = (merged_df[f'pred_{target_stat}'] - model_info["std"]).round()
                merged_df[f'upper_bound_{target_stat}'] = (merged_df[f'pred_{target_stat}'] + model_info["std"]).round()

                target_stat_columns.append(f'pred_{target_stat}')
            
            display(merged_df[["lineup_name", 'player_name', 'team_name', 'opp_name'] + target_stat_columns])
        except Exception as e:
            print(e)
            continue

In [105]:
get_games_info()

,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,DeAaron Fox,DeAaron Fox,Sacramento Kings,Los Angeles Lakers,21.877803,5.303587,6.595053
1,Malik Monk,Malik Monk,Sacramento Kings,Los Angeles Lakers,18.334292,4.350706,6.755698
2,DeMar DeRozan,DeMar DeRozan,Sacramento Kings,Los Angeles Lakers,21.297452,3.550922,3.802493
3,Keegan Murray,Keegan Murray,Sacramento Kings,Los Angeles Lakers,10.906095,6.724783,1.178323
4,D. Sabonis,Domantas Sabonis,Sacramento Kings,Los Angeles Lakers,20.347900,12.015350,5.846626
5,Austin Reaves,Austin Reaves,Los Angeles Lakers,Sacramento Kings,16.064107,3.198710,4.546049
6,Max Christie,Max Christie,Los Angeles Lakers,Sacramento Kings,9.160261,3.777288,1.814849
7,Rui Hachimura,Rui Hachimura,Los Angeles Lakers,Sacramento Kings,11.386312,5.433287,1.875851
8,LeBron James,LeBron James,Los Angeles Lakers,Sacramento Kings,21.628582,6.980497,8.285080
9,Anthony Davis,Anthony Davis,Los Angeles Lakers,Sacramento Kings,24.271274,11.458569,4.033124


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,Anthony Black,Anthony Black,Orlando Magic,Miami Heat,8.760680,3.045489,2.915386
1,K. Caldwell-Pope,Kentavious Caldwell-Pope,Orlando Magic,Miami Heat,9.941583,2.209555,1.870256
2,T. da Silva,Tristan da Silva,Orlando Magic,Miami Heat,8.551130,3.317982,1.309189
3,W. Carter,Wendell Carter,Orlando Magic,Miami Heat,7.303160,7.129058,2.280847
4,Goga Bitadze,Goga Bitadze,Orlando Magic,Miami Heat,10.121040,8.717880,2.440581
5,Tyler Herro,Tyler Herro,Miami Heat,Orlando Magic,23.416903,5.225475,4.526003
6,D. Robinson,Duncan Robinson,Miami Heat,Orlando Magic,12.214924,2.524796,2.723136
7,Dru Smith,Dru Smith,Miami Heat,Orlando Magic,6.546496,3.143775,1.977764
8,Nikola Jovic,Nikola Jovic,Miami Heat,Orlando Magic,8.313896,3.081412,1.021336
9,Bam Adebayo,Bam Adebayo,Miami Heat,Orlando Magic,15.394420,9.970907,6.011569


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,Vit Krejci,Vit Krejci,Atlanta Hawks,Memphis Grizzlies,3.747341,2.466586,1.483447
1,Dyson Daniels,Dyson Daniels,Atlanta Hawks,Memphis Grizzlies,10.416022,5.595296,3.624977
2,Z. Risacher,Zaccharie Risacher,Atlanta Hawks,Memphis Grizzlies,11.083402,2.957844,0.895346
3,Jalen Johnson,Jalen Johnson,Atlanta Hawks,Memphis Grizzlies,18.669936,8.858718,5.584688
4,Clint Capela,Clint Capela,Atlanta Hawks,Memphis Grizzlies,7.563779,10.652282,1.301650
5,Scotty Pippen,Scotty Pippen,Memphis Grizzlies,Atlanta Hawks,7.666248,2.971670,3.857403
6,Desmond Bane,Desmond Bane,Memphis Grizzlies,Atlanta Hawks,12.950938,4.660691,5.440971
7,Jaylen Wells,Jaylen Wells,Memphis Grizzlies,Atlanta Hawks,10.076844,3.542743,1.695867
8,Jaren Jackson,Jaren Jackson,Memphis Grizzlies,Atlanta Hawks,19.269657,5.739041,1.978800
9,Zach Edey,Zach Edey,Memphis Grizzlies,Atlanta Hawks,10.872490,7.812528,1.966305


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,Ben Simmons,Ben Simmons,Brooklyn Nets,Utah Jazz,6.081638,4.459746,6.286351
1,Keon Johnson,Keon Johnson,Brooklyn Nets,Utah Jazz,8.988221,3.054672,1.615047
2,Jalen Wilson,Jalen Wilson,Brooklyn Nets,Utah Jazz,10.297136,3.227706,1.818572
3,C. Johnson,Cameron Johnson,Brooklyn Nets,Utah Jazz,17.696829,4.627858,3.463493
4,Nic Claxton,Nic Claxton,Brooklyn Nets,Utah Jazz,10.179243,6.432008,1.951947
5,I. Collier,Isaiah Collier,Utah Jazz,Brooklyn Nets,3.600662,1.404732,3.228106
6,Collin Sexton,Collin Sexton,Utah Jazz,Brooklyn Nets,16.062511,2.916324,4.041778
7,L. Markkanen,Lauri Markkanen,Utah Jazz,Brooklyn Nets,20.351733,6.105140,2.125108
8,John Collins,John Collins,Utah Jazz,Brooklyn Nets,15.846226,7.307519,2.843594
9,W. Kessler,Walker Kessler,Utah Jazz,Brooklyn Nets,11.538955,9.482385,2.125108


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,D. Murray,Dejounte Murray,New Orleans Pelicans,New York Knicks,17.052426,5.592912,6.716673
1,CJ McCollum,CJ McCollum,New Orleans Pelicans,New York Knicks,20.990919,3.990567,3.749071
2,Trey Murphy,Trey Murphy III,New Orleans Pelicans,New York Knicks,18.412813,4.086971,2.316506
3,Herbert Jones,Herbert Jones,New Orleans Pelicans,New York Knicks,11.622285,3.857402,2.428267
4,Yves Missi,Yves Missi,New Orleans Pelicans,New York Knicks,11.661111,9.151911,1.011737
5,Jalen Brunson,Jalen Brunson,New York Knicks,New Orleans Pelicans,22.336425,3.467805,6.702517
6,Mikal Bridges,Mikal Bridges,New York Knicks,New Orleans Pelicans,19.278273,3.560214,3.231706
7,Josh Hart,Josh Hart,New York Knicks,New Orleans Pelicans,11.341974,6.775277,4.168714
8,OG Anunoby,OG Anunoby,New York Knicks,New Orleans Pelicans,14.257451,4.560132,2.068378
9,K. Towns,Karl-Anthony Towns,New York Knicks,New Orleans Pelicans,21.123010,14.622371,4.643992


...

,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,AJ Green,AJ Green,Milwaukee Bucks,Washington Wizards,7.657927,2.783286,1.619738
1,Andre Jackson,Andre Jackson,Milwaukee Bucks,Washington Wizards,5.499050,4.092980,1.619738
2,T. Prince,Taurean Prince,Milwaukee Bucks,Washington Wizards,7.519070,3.183991,1.625614
3,Bobby Portis,Bobby Portis,Milwaukee Bucks,Washington Wizards,11.421674,7.265713,2.146527
4,Brook Lopez,Brook Lopez,Milwaukee Bucks,Washington Wizards,9.107744,4.537377,1.517071
5,Jordan Poole,Jordan Poole,Washington Wizards,Milwaukee Bucks,20.678988,2.656853,5.899185
6,C. Carrington,Carlton Carrington,Washington Wizards,Milwaukee Bucks,8.796814,4.791517,3.118684
7,B. Coulibaly,Bilal Coulibaly,Washington Wizards,Milwaukee Bucks,13.355774,4.620872,4.304185
8,J. Champagnie,Justin Champagnie,Washington Wizards,Milwaukee Bucks,11.911313,6.153441,1.436637
9,A. Sarr,Alexandre Sarr,Washington Wizards,Milwaukee Bucks,12.784660,7.048444,2.030498


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,Mike Conley,Mike Conley,Minnesota Timberwolves,Golden State Warriors,10.011408,3.076687,4.044160
1,A. Edwards,Anthony Edwards,Minnesota Timberwolves,Golden State Warriors,21.884532,5.285740,4.460322
2,J. McDaniels,Jaden McDaniels,Minnesota Timberwolves,Golden State Warriors,9.710302,4.846411,1.821075
3,Julius Randle,Julius Randle,Minnesota Timberwolves,Golden State Warriors,18.406456,6.401254,3.862784
4,Rudy Gobert,Rudy Gobert,Minnesota Timberwolves,Golden State Warriors,8.844741,9.611292,2.089728
5,Stephen Curry,Stephen Curry,Golden State Warriors,Minnesota Timberwolves,21.358081,4.677353,6.035764
6,D. Schroder,Dennis Schroder,Golden State Warriors,Minnesota Timberwolves,18.582695,3.185464,6.590534
7,A. Wiggins,Andrew Wiggins,Golden State Warriors,Minnesota Timberwolves,17.520249,4.129696,2.329650
8,D. Green,Draymond Green,Golden State Warriors,Minnesota Timberwolves,9.410986,5.875108,5.618433
9,Kevon Looney,Kevon Looney,Golden State Warriors,Minnesota Timberwolves,4.939984,6.575836,1.818891


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,S. Dinwiddie,Spencer Dinwiddie,Dallas Mavericks,Los Angeles Clippers,13.405189,2.734002,4.373967
1,Kyrie Irving,Kyrie Irving,Dallas Mavericks,Los Angeles Clippers,21.255221,4.708115,5.332230
2,Klay Thompson,Klay Thompson,Dallas Mavericks,Los Angeles Clippers,14.587514,3.883876,1.879134
3,P. Washington,P.J. Washington,Dallas Mavericks,Los Angeles Clippers,12.881455,6.940340,3.285912
4,Dereck Lively,Dereck Lively II,Dallas Mavericks,Los Angeles Clippers,8.582059,8.115782,2.741462
5,James Harden,James Harden,Los Angeles Clippers,Dallas Mavericks,24.546111,4.163786,7.497911
6,Kris Dunn,Kris Dunn,Los Angeles Clippers,Dallas Mavericks,6.226969,3.070530,2.141558
7,Norman Powell,Norman Powell,Los Angeles Clippers,Dallas Mavericks,19.346380,3.190700,3.045226
8,Derrick Jones,Derrick Jones,Los Angeles Clippers,Dallas Mavericks,9.613664,2.999862,1.170479
9,Ivica Zubac,Ivica Zubac,Los Angeles Clippers,Dallas Mavericks,13.799360,10.501556,2.365041


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,Chris Paul,Chris Paul,San Antonio Spurs,Portland Trail Blazers,8.393106,4.433961,7.559994
1,Devin Vassell,Devin Vassell,San Antonio Spurs,Portland Trail Blazers,15.653070,2.722823,2.717904
2,H. Barnes,Harrison Barnes,San Antonio Spurs,Portland Trail Blazers,10.992901,3.289402,1.803927
3,Jeremy Sochan,Jeremy Sochan,San Antonio Spurs,Portland Trail Blazers,13.479603,7.742964,2.855070
4,V. Wembanyama,Victor Wembanyama,San Antonio Spurs,Portland Trail Blazers,26.333370,9.627431,6.046446
5,A. Simons,Anfernee Simons,Portland Trail Blazers,San Antonio Spurs,19.722791,3.375654,5.723969
6,S. Sharpe,Shaedon Sharpe,Portland Trail Blazers,San Antonio Spurs,17.175176,4.237489,3.114523
7,T. Camara,Toumani Camara,Portland Trail Blazers,San Antonio Spurs,9.571248,5.390025,1.959419
8,Jerami Grant,Jerami Grant,Portland Trail Blazers,San Antonio Spurs,15.019053,4.023223,2.934391
9,Deandre Ayton,Deandre Ayton,Portland Trail Blazers,San Antonio Spurs,13.528948,8.823602,1.900681


,lineup_name,player_name,team_name,opp_name,pred_pts,pred_reb,pred_ast
0,Tyus Jones,Tyus Jones,Phoenix Suns,Detroit Pistons,12.021836,2.556001,5.881566
1,Bradley Beal,Bradley Beal,Phoenix Suns,Detroit Pistons,18.077999,4.325887,2.940550
2,Royce ONeale,Royce ONeale,Phoenix Suns,Detroit Pistons,11.367721,5.193428,2.197091
3,Kevin Durant,Kevin Durant,Phoenix Suns,Detroit Pistons,24.017436,6.014984,4.931501
4,Jusuf Nurkic,Jusuf Nurkic,Phoenix Suns,Detroit Pistons,8.669263,8.917228,1.656104
5,C. Cunningham,Cade Cunningham,Detroit Pistons,Phoenix Suns,22.554097,6.984387,10.060706
6,Jaden Ivey,Jaden Ivey,Detroit Pistons,Phoenix Suns,14.490245,3.063572,4.282032
7,Tim Hardaway,Tim Hardaway,Detroit Pistons,Phoenix Suns,9.800008,3.078681,1.303718
8,Tobias Harris,Tobias Harris,Detroit Pistons,Phoenix Suns,14.541138,5.631195,2.002348
9,Jalen Duren,Jalen Duren,Detroit Pistons,Phoenix Suns,8.552506,6.723046,2.517083
